In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [6]:
(sqrt(2)-I).minpoly('xx')

xx^4 - 2*xx^2 + 9

In [7]:
var('xx')
Q.<rad2_minus_i> = NumberField(xx^4 - 2*xx^2 + 9)

In [27]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})

In [28]:
%time adm_SE1 = SE1.find_admissible_projectivities()
len(adm_SE1)

CPU times: user 1.69 s, sys: 225 ms, total: 1.92 s
Wall time: 11.8 s


576

In [29]:
%time simm_SE1 = SE1.find_simmetries(adm_SE1)
len(simm_SE1)

CPU times: user 1.9 s, sys: 165 ms, total: 2.07 s
Wall time: 6.3 s


2

In [30]:
lines_perms = []
for simm in simm_SE1:
    lines_perms.append(Permutation(SE1.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(2, 'C2')

In [31]:
adm_perms = [from_labels_to_perm(label) for label in SE1.find_admissible_permutations()]
M2_perm = lines_perms_group.gens_small()[0]

In [32]:
adm_perms_group = PermutationGroup(adm_perms)

In [33]:
# find permutation of order 4 whose square is M2_perm
possible_perms = []
for perm in adm_perms_group:
    if perm^2 == M2_perm and perm.order() == 4:
        possible_perms.append(perm)
len(possible_perms)

12

In [34]:
possible_L_sets = [get_permuted_L_set(perm) for perm in possible_perms]
L_set_base = ['E1', 'G4', 'E2', 'G3', 'E3']

In [35]:
possible_projs = [find_projectivity(SE1.get_L_set_in_plucker(L_set_base), SE1.get_L_set_in_plucker(L_set)) for L_set in possible_L_sets]
M2 = [mat for mat in simm_SE1 if mat[0][0]!=1][0]
identity_list = [0 for _ in range(16)]
for i in {0,5,10,15}:
    identity_list[i]=1
possible_projs_squared = [proj^2 for proj in possible_projs]    

In [46]:
proj = possible_projs_squared[-1]

In [47]:
conds = list(set(matrix([proj.list(), M2.list()]).minors(2)))
conds = [remove_sing_factors(el, SE1.sing_locus) for el in conds if el !=0]

In [48]:
mon = ((x+y+z+t)^3).monomials()
sost = change_coord(proj)
new_cubic = remove_sing_factors(SE1.eqn.subs(sost), SE1.sing_locus)    
minor = list(set(matrix([[new_cubic.coefficient(mn) for mn in mon], [SE1.eqn.coefficient(mn) for mn in mon]]).minors(2)))
minor = [remove_sing_factors(el, SE1.sing_locus) for el in minor if el !=0]

In [49]:
final_conds = list(set(conds+minor))

In [85]:
gb = [remove_sing_factors(el, SE1.sing_locus) for el in P.ideal(final_conds).groebner_basis()]
gbf = gb+[f]

In [86]:
ideals = P.ideal(gbf).radical().primary_decomposition()
ideals

[Ideal (f, c) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field,
 Ideal (f, c + 5*d - 10*e, 5*d^2 - 22*d*e + 25*e^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field]

In [87]:
%%time
sat_ideal = P.ideal(gb).saturation(P.ideal(f))[0]
ideals += sat_ideal.radical().primary_decomposition()

CPU times: user 782 ms, sys: 4.07 ms, total: 786 ms
Wall time: 789 ms


In [88]:
ideals = list(set(ideals))
ideals = [ideal for ideal in ideals if SE1.sing_locus.value() not in ideal]

In [89]:
ideals

[Ideal (f, c + 5*d - 10*e, 5*d^2 - 22*d*e + 25*e^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field,
 Ideal (3*c^2*d - 7*c^2*e + c*d*e - 5*c*e^2 + c^2*f + c*d*f - 2*c*e*f - d*e*f - 3*e^2*f - 5*c*f^2 - 3*e*f^2, 12*c^3 + 6*c*d^2 + 20*c^2*e - 8*c*d*e - 3*d^2*e + 34*c*e^2 - 6*d*e^2 - 15*e^3 + 16*c^2*f - 8*c*d*f - 3*d^2*f - 44*c*e*f - 4*d*e*f + 27*e^2*f + 46*c*f^2 - 6*d*f^2 + 15*e*f^2 - 15*f^3, 64*c*d*e^2 - 5*d^2*e^2 - 160*c*e^3 - 10*d*e^3 - 25*e^4 - 24*c*d^2*f - 32*c^2*e*f + 12*c*d*e*f + 6*d^2*e*f + 76*c*e^2*f - 18*d*e^2*f - 64*e^3*f - 8*c^2*f^2 - 60*c*d*f^2 + 7*d^2*f^2 + 48*c*e*f^2 + 30*d*e*f^2 - 50*e^2*f^2 - 44*c*f^3 + 14*d*f^3 + 64*e*f^3 + 35*f^4, 64*c^2*e^2 + 25*d^2*e^2 + 160*c*e^3 + 50*d*e^3 + 125*e^4 + 24*c*d^2*f + 96*c^2*e*f + 68*c*d*e*f + 18*d^2*e*f + 36*c*e^2*f + 58*d*e^2*f - 16*e^3*f + 40*c^2*f^2 + 108*c*d*f^2 + 13*d^2*f^2 + 80*c*e*f^2 + 42*d*e*f^2 + 10*e^2*f^2 + 124*c*f^3 + 26*d*f^3 + 16*e*f^3 + 65*f^4, 96*c*d^2*e - 33*d^2*e^2 - 576*c*e^3 -

In [63]:
mat = matrix(4,4,[ideals[1].reduce(el) for el in proj.list()])

In [68]:
list(set([ideals[1].reduce(el) for el in matrix([mat.list(), M2.list()]).minors(2)]))

[0]

In [69]:
SE1.reduce(ideals[1])

-x^2*z*c^2 - x*z^2*c^2 - x*y*t*c^2 + y*z*t*c^2 + 2*y*t^2*c^2 + x*y*z*c*d - x*z^2*c*d + y^2*t*c*d - y*z*t*c*d - x^2*z*c*e + x*y*z*c*e + x*y*t*c*e - y^2*t*c*e - x^2*z*c*f + x*y*z*c*f + x*y*t*c*f - y^2*t*c*f + 2*x^2*y*e*f - 2*x*y^2*e*f - x^2*z*e*f + x*z^2*e*f - x*y*t*e*f + y*z*t*e*f

In [59]:
SE1

-x^2*z*c^2 - x*z^2*c^2 - x*y*t*c^2 + y*z*t*c^2 + 2*y*t^2*c^2 + x*y*z*c*d - x*z^2*c*d + y^2*t*c*d - y*z*t*c*d - x^2*z*c*e + x*y*z*c*e + x*y*t*c*e - y^2*t*c*e - x^2*z*c*f + x*y*z*c*f + x*y*t*c*f - y^2*t*c*f + 2*x^2*y*e*f - 2*x*y^2*e*f - x^2*z*e*f + x*z^2*e*f - x*y*t*e*f + y*z*t*e*f

In [72]:
for gen in ideals[1].gens():
    print(gen)
    print()

3*c^2*d - 7*c^2*e + c*d*e - 5*c*e^2 + c^2*f + c*d*f - 2*c*e*f - d*e*f - 3*e^2*f - 5*c*f^2 - 3*e*f^2

12*c^3 + 6*c*d^2 + 20*c^2*e - 8*c*d*e - 3*d^2*e + 34*c*e^2 - 6*d*e^2 - 15*e^3 + 16*c^2*f - 8*c*d*f - 3*d^2*f - 44*c*e*f - 4*d*e*f + 27*e^2*f + 46*c*f^2 - 6*d*f^2 + 15*e*f^2 - 15*f^3

64*c*d*e^2 - 5*d^2*e^2 - 160*c*e^3 - 10*d*e^3 - 25*e^4 - 24*c*d^2*f - 32*c^2*e*f + 12*c*d*e*f + 6*d^2*e*f + 76*c*e^2*f - 18*d*e^2*f - 64*e^3*f - 8*c^2*f^2 - 60*c*d*f^2 + 7*d^2*f^2 + 48*c*e*f^2 + 30*d*e*f^2 - 50*e^2*f^2 - 44*c*f^3 + 14*d*f^3 + 64*e*f^3 + 35*f^4

64*c^2*e^2 + 25*d^2*e^2 + 160*c*e^3 + 50*d*e^3 + 125*e^4 + 24*c*d^2*f + 96*c^2*e*f + 68*c*d*e*f + 18*d^2*e*f + 36*c*e^2*f + 58*d*e^2*f - 16*e^3*f + 40*c^2*f^2 + 108*c*d*f^2 + 13*d^2*f^2 + 80*c*e*f^2 + 42*d*e*f^2 + 10*e^2*f^2 + 124*c*f^3 + 26*d*f^3 + 16*e*f^3 + 65*f^4

96*c*d^2*e - 33*d^2*e^2 - 576*c*e^3 - 66*d*e^3 - 165*e^4 - 24*c*d^2*f - 160*c^2*e*f + 28*c*d*e*f - 18*d^2*e*f - 68*c*e^2*f - 234*d*e^2*f - 96*e^3*f - 104*c^2*f^2 - 332*c*d*f^2 + 27*d^2*

In [73]:
gen = ideals[1].gens()[0]